In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import os

# proxyの設定．
# keras.datasetsでは，datasetを直接ダウンロードするので，学内マシンからは通常必要．
os.environ["http_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["https_proxy"] = "http://proxy.uec.ac.jp:8080/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"      # "0":GPU0, "1":GPU1, "0,1":GPUを2つとも使用

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
#googlenetの最後三つのavgpool,dropout,fcレイヤーの削除方法を見つからないため、vgg19を使う
import torchvision.models as models
from PIL import Image
import torchvision.transforms as transforms
import torch

from misc_functions import get_example_params, convert_to_grayscale, save_gradient_images

In [34]:
import os
from PIL import Image

import torch
from torch.optim import SGD
from torchvision import models

from misc_functions import preprocess_image, recreate_image, save_image


class DeepDream():
    """
        Produces an image that minimizes the loss of a convolution
        operation for a specific layer and filter
    """
    def __init__(self, model, selected_layer, selected_filter, im_path):
        self.model = model
        self.model.eval()
        self.selected_layer = selected_layer
        self.selected_filter = selected_filter
        self.conv_output = 0
        # Generate a random image
        self.created_image = Image.open(im_path).convert('RGB')
        # Hook the layers to get result of the convolution
        self.hook_layer()
        # Create the folder to export images if not exists
        if not os.path.exists('generated'):
            os.makedirs('generated')

    def hook_layer(self):
        def hook_function(module, grad_in, grad_out):
            # Gets the conv output of the selected filter (from selected layer)
            self.conv_output = grad_out[0, self.selected_filter]

        # Hook the selected layer
        self.model[self.selected_layer].register_forward_hook(hook_function)

    def dream(self):
        # Process image and return variable
        self.processed_image = preprocess_image(self.created_image, False)
        # Define optimizer for the image
        # Earlier layers need higher learning rates to visualize whereas layer layers need less
        optimizer = SGD([self.processed_image], lr=12,  weight_decay=1e-4)
        for i in range(1, 551):
            optimizer.zero_grad()
            # Assign create image to a variable to move forward in the model
            x = self.processed_image
            x = x.cuda()
            for index, layer in enumerate(self.model):
                # Forward
                x = layer(x)
                # Only need to forward until we the selected layer is reached
                if index == self.selected_layer:
                    break
            # Loss function is the mean of the output of the selected layer/filter
            # We try to minimize the mean of the output of that specific filter
            loss = -torch.mean(self.conv_output)
            if i % 50 == 0:
                print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data.cpu().numpy()))
            # Backward
            loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image)
            # Save image every 20 iteration
            if i % 50 == 0:
                print(self.created_image.shape)
                im_path = 'generated/' + 'fish_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)


In [35]:
cnn_layer = 34
filter_pos = 284

im_path = 'goldfish.jpg'
# Fully connected layer is not needed
pretrained_model = models.vgg19(pretrained=True).features
pretrained_model = pretrained_model.cuda()
dd = DeepDream(pretrained_model, cnn_layer, filter_pos, im_path)
# This operation can also be done without Pytorch hooks
# See layer visualisation for the implementation without hooks
dd.dream()

Iteration: 50 Loss: -38.22
(650, 980, 3)
Iteration: 100 Loss: -72.91
(650, 980, 3)
Iteration: 150 Loss: -99.46
(650, 980, 3)
Iteration: 200 Loss: -121.35
(650, 980, 3)
Iteration: 250 Loss: -140.38
(650, 980, 3)
Iteration: 300 Loss: -157.55
(650, 980, 3)
Iteration: 350 Loss: -173.55
(650, 980, 3)
Iteration: 400 Loss: -188.68
(650, 980, 3)
Iteration: 450 Loss: -203.08
(650, 980, 3)
Iteration: 500 Loss: -216.85
(650, 980, 3)
Iteration: 550 Loss: -230.03
(650, 980, 3)
